In [23]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests
from io import StringIO

In [7]:
url = "https://tools.morningstar.co.uk/api/rest.svc/klr5zyak8x/security/screener"
params = {
    'page': 1,
    'pageSize': 50,
    'sortOrder': 'LegalName asc',
    'outputType': 'json',
    'version': 1,
    'languageId': 'es-ES',
    'currencyId': 'EUR',
    'universeIds': 'FOESP$$ALL',
    'securityDataPoints': 'SecId|Name|PriceCurrency|TenforeId|LegalName|ClosePrice|Yield_M12|CategoryName|Medalist_RatingNumber|StarRatingM255|SustainabilityRank|ReturnD1|ReturnW1|ReturnM1|ReturnM3|ReturnM6|ReturnM0|ReturnM12|ReturnM36|ReturnM60|ReturnM120|FeeLevel|ManagerTenure|MaxDeferredLoad|InitialPurchase|FundTNAV|EquityStyleBox|BondStyleBox|AverageMarketCapital|AverageCreditQualityCode|EffectiveDuration|MorningstarRiskM255|AlphaM36|BetaM36|R2M36|StandardDeviationM36|SharpeM36|TrackRecordExtension',
    'filters': '',
    'term': '',
    'subUniverseId': ''
}

headers = {
#    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'User-Agent': UserAgent().random,
    'Accept': 'application/json, text/plain, */*',
}

# Realizar la solicitud GET
response = requests.get(url, headers=headers, params=params)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    # Convertir la respuesta a JSON
    data = response.json()
    #print(data)
else:
    print("Error:", response.status_code)


### Descripción de las Columnas del DataFrame de Fondos de Morningstar

- **`SecId`**: Identificador único del fondo en la plataforma de Morningstar.
- **`Name`**: Nombre comercial del fondo.
- **`PriceCurrency`**: La moneda en la que se expresa el precio del fondo.
- **`LegalName`**: Nombre legal del fondo, como aparece en los documentos oficiales.
- **`ClosePrice`**: Precio de cierre del fondo en la última sesión contable.
- **`Yield_M12`**: Rendimiento del fondo en los últimos 12 meses.
- **`CategoryName`**: Nombre de la categoría Morningstar a la que pertenece el fondo.
- **`ReturnD1`**: Retorno del fondo en el último día.
- **`ReturnW1`**: Retorno del fondo en la última semana.
- **`ReturnM1`**: Retorno del fondo en el último mes.
- **`ReturnM3`**: Retorno del fondo en los últimos 3 meses.
- **`ReturnM6`**: Retorno del fondo en los últimos 6 meses.
- **`ReturnM0`**: Retorno del fondo en el mes actual.
- **`ReturnM12`**: Retorno del fondo en los últimos 12 meses.
- **`ReturnM36`**: Retorno del fondo en los últimos 36 meses.
- **`ReturnM60`**: Retorno del fondo en los últimos 60 meses.
- **`ManagerTenure`**: Tiempo que el gestor actual ha estado al frente del fondo, en años.
- **`MaxDeferredLoad`**: Comisión máxima por suscripción diferida que se cobra al vender el fondo.
- **`StandardDeviationM36`**: Desviación estándar de los retornos del fondo en los últimos 36 meses, una medida de volatilidad.
- **`SharpeM36`**: Ratio de Sharpe en los últimos 36 meses, que mide la rentabilidad ajustada al riesgo.
- **`TrackRecordExtension`**: Indica si el fondo ha sido prolongado o extendido más allá de su vida útil prevista inicialmente.
- **`TenforeId`**: Identificador de Tenfore para el fondo.
- **`Medalist_RatingNumber`**: Clasificación numérica de los analistas de Morningstar para el fondo.
- **`StarRatingM255`**: Calificación por estrellas de Morningstar para el fondo, basada en su rendimiento ajustado al riesgo pasado.
- **`SustainabilityRank`**: Clasificación de sostenibilidad del fondo, evaluando factores como impacto ambiental, social y de gobernanza.
- **`FeeLevel`**: Nivel de las tasas de gestión y otros gastos asociados al fondo.
- **`InitialPurchase`**: Monto mínimo inicial requerido para invertir en el fondo.
- **`FundTNAV`**: Valor total neto de los activos del fondo.
- **`EquityStyleBox`**: Representación gráfica de la orientación de inversión del fondo en acciones, basada en el tamaño del mercado y el estilo de valor/crecimiento.
- **`AverageMarketCapital`**: Capitalización de mercado media de las inversiones del fondo.
- **`MorningstarRiskM255`**: Evaluación del riesgo del fondo por Morningstar basada en los últimos 255 días.
- **`AlphaM36`**: Alpha del fondo en los últimos 36 meses, que mide su rendimiento relativo ajustado al riesgo frente a su índice de referencia.
- **`BetaM36`**: Beta del fondo en los últimos 36 meses, que mide la sensibilidad del fondo a los movimientos del mercado.
- **`R2M36`**: Coeficiente de determinación (R-cuadrado) para los últimos 36 meses, que mide la proporción de variabilidad en los retornos del fondo que puede ser explicada por los movimientos del índice de referencia.
- **`BondStyleBox`**: Representación gráfica de la orientación de inversión del fondo en bonos, basada en la duración y la calidad del crédito.
- **`AverageCreditQualityCode`**: Código que representa la calidad de crédito promedio de los bonos en el portafolio del fondo.
- **`EffectiveDuration`**: Duración efectiva de la cartera de bonos del fondo, una medida


In [8]:
fondos_df = pd.DataFrame(data['rows'])


(50, 38)

In [63]:
#fondos_df.to_csv('fondos_descargados.csv', index=False)

In [9]:
tickers = list(fondos_df['SecId'])
nombres_fondos = list(fondos_df['LegalName'])

In [10]:
category_mapping = {
    'RV': 'RV',
    'RF': 'RF',
    'Mixtos': 'Mixtos',
    'Alternativo': 'Alternativo',
    'Inmobiliario': 'Inmobiliario',
    'Mercado Monetario': 'Mercado Monetario',
    'Otros': 'Otros',
    'Materias Primas': 'Materias Primas',
    'Fecha Objetivo': 'Fecha Objetivo',
    'Capital Protegido': 'Capital Protegido',
    'EUR Deuda Subordinada': 'EUR Deuda',
    'Garantizados': 'Garantizados'
}

In [11]:
# Aplicar el mapeo usando un bucle para optimizar la asignación de valores
for key, value in category_mapping.items():
    fondos_df.loc[fondos_df['CategoryName'].str.startswith(key), 'Cat'] = value

In [21]:
def fetch_fund_data(sec_id):
    params = {'id': sec_id}
    headers = {'User-Agent': 'Mozilla/5.0'}  # Ajusta según necesites
    url = 'https://www.morningstar.es/es/funds/snapshot/snapshot.aspx'
    
    # Realizar la solicitud
    response = requests.get(url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar todas las tablas en el documento y extraer las últimas palabras de sus clases
    tables = soup.find_all('table')
    dataframes = {}

    for table in tables:
        class_names = ' '.join(table.get('class', []))
        words = class_names.split()
        last_word = words[-1] if words else 'Unnamed'

        # Solo procesar tablas si la última palabra es relevante y no está en la lista de exclusión
        if last_word not in ['Unnamed', 'snapshot']:
            html_content = str(table)
            # Utilizar StringIO para simular un archivo en memoria
            df = pd.read_html(StringIO(html_content))[0]
            dataframes[last_word] = df

    return dataframes

In [ ]:
fondos_df['SecId']

In [27]:
RV_funds = fondos_df.loc[fondos_df['Cat'] == 'RV', 'SecId']
# Diccionario para almacenar DataFrames por SecId
all_funds_data = {}

for sec_id in RV_funds:
    try:
        all_funds_data[sec_id] = fetch_fund_data(sec_id)
        #print(f"Datos obtenidos para SecId {sec_id}")
    except Exception as e:
        print(f"Error obteniendo datos para SecId {sec_id}: {e}")

In [43]:
list(all_funds_data.get('F00001COTM').keys())   

['snapshotTitleTable',
 'overviewPerformanceTable',
 'overviewCalenderYearReturnsTable',
 'overviewKeyStatsTable',
 'overviewObjectiveTable',
 'TrailingReturnsOverview',
 'overviewTrailingReturnsTable',
 'FundManagersOverview',
 'snapshotTable',
 'overviewBenchmarkTable2Cols',
 'overviewPortfolioTable',
 'overviewPortfolioEquityStyleTable',
 'overviewAssetAllocationTable',
 'overviewTopRegionsTable',
 'overviewTopSectorsTable',
 'overviewTopHoldingsTable']

In [14]:
params = {'id': fondos_df['SecId'][1]}

html = requests.get('https://www.morningstar.es/es/funds/snapshot/snapshot.aspx', params=params, headers=headers).text
soup = BeautifulSoup(html, 'html.parser')


In [50]:
# Encontrar todas las tablas en el documento
tables = soup.find_all('table')

# Lista para almacenar las últimas palabras de los nombres de las tablas
last_word_names = []

for table in tables:
    # Obtener el atributo 'class', convertirlo en una cadena y separarlo por espacios
    class_names = ' '.join(table.get('class', []))
    # Dividir la cadena por espacios para obtener las palabras individuales
    words = class_names.split()
    # Tomar la última palabra de la lista de palabras, si existe
    if words:
        last_word = words[-1]
        last_word_names.append(last_word)
    else:
        # Si no hay palabras, agregar un marcador para indicar que no hay nombre
        last_word_names.append('Unnamed')

# Filtrar la lista para excluir los que contienen 'snapshot' o son 'Unnamed'
filtered_tables = [name for name in last_word_names if 'snapshot' not in name and name != 'Unnamed']

# Imprimir los nombres filtrados
print(filtered_tables)

['overviewPerformanceTable', 'overviewCalenderYearReturnsTable', 'overviewKeyStatsTable', 'overviewObjectiveTable', 'TrailingReturnsOverview', 'overviewTrailingReturnsTable', 'overviewKeyStatsTable', 'FundManagersOverview', 'overviewBenchmarkTable2Cols', 'overviewObjectiveTable', 'overviewPortfolioTable', 'overviewPortfolioTable', 'overviewPortfolioTable', 'overviewPortfolioEquityStyleTable', 'overviewAssetAllocationTable', 'overviewTopRegionsTable', 'overviewTopSectorsTable', 'overviewTopHoldingsTable']


In [52]:
tabla_html = soup.find('table', class_='overviewKeyStatsTable')
tabla_html

<table border="0" class="snapshotTextColor snapshotTextFontStyle snapshotTable overviewKeyStatsTable"><tr><td class="titleBarHeading" colspan="3">Estadística Rápida</td></tr><tr><td class="line heading">VL<span class="heading"><br/>18/04/2024</span></td><td class="line"> </td><td class="line text">USD 148,51</td></tr><tr><td class="line heading">Cambio del día</td><td class="line"> </td><td class="line text">-0,72%
                      </td></tr><tr><td class="line heading">Categoría Morningstar™</td><td class="line"> </td><td class="line value text"><a href="/es/fundquickrank/default.aspx?category=EUCA000558" style="width:100%!important;">RV Global  Cap. Flexible</a></td></tr><tr><td class="line heading">ISIN</td><td class="line"> </td><td class="line text">LU1785301273</td></tr><tr><td class="line heading">Patrimonio (Mil)<span class="heading"><br/>18/04/2024</span></td><td class="line"> </td><td class="line text">USD 573,04</td></tr><tr><td class="line heading">Patrimonio Clase (Mi

In [57]:
if tabla_html:
    # Convertir la tabla HTML en DataFrame
    tabla_df = pd.read_html(str(tabla_html))[0].iloc[1:, [0,2]] # read_html devuelve una lista de DataFrames, seleccionamos el primero

else:
    print("No se encontró la tabla 'overviewPerformanceTable'")
    
tabla_df

/tmp/ipykernel_631/3860323332.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tabla_df = pd.read_html(str(tabla_html))[0].iloc[1:, [0,2]] # read_html devuelve una lista de DataFrames, seleccionamos el primero


,0,2
1,VL 18/04/2024,"USD 148,51"
2,Cambio del día,"-0,72%"
3,Categoría Morningstar™,RV Global Cap. Flexible
4,ISIN,LU1785301273
5,Patrimonio (Mil) 18/04/2024,"USD 573,04"
6,Patrimonio Clase (Mil) 18/04/2024,"USD 8,69"
7,Comisión Máx. Suscripción,-
8,Gastos Corrientes 30/06/2023,"1,00%"


In [59]:
import re
def clean_convert(value):
    if isinstance(value, str):
        # Usar expresiones regulares para eliminar 'USD ' y '%' y convertir ',' en '.'
        value = re.sub(r'USD\s*', '', value)
        value = value.replace('%', '').replace(',', '.')
        try:
            # Intentar convertir a float
            return float(value)
        except ValueError:
            # Si hay error, regresa el valor original limpio
            return value
    return value

# Aplicar la función a cada elemento del DataFrame
tabla_df[2] = tabla_df[2].apply(clean_convert)

# Mostrar el DataFrame modificado
print("\nDataFrame Modificado:")
tabla_df


DataFrame Modificado:


,0,2
1,VL 18/04/2024,148.51
2,Cambio del día,-0.72
3,Categoría Morningstar™,RV Global Cap. Flexible
4,ISIN,LU1785301273
5,Patrimonio (Mil) 18/04/2024,573.04
6,Patrimonio Clase (Mil) 18/04/2024,8.69
7,Comisión Máx. Suscripción,-
8,Gastos Corrientes 30/06/2023,1.0


In [56]:
def clean_convert(value):
    if isinstance(value, str):
        # Eliminar las unidades y convertir a número
        value = value.replace('USD ', '').replace('%', '').replace(',', '.')
        try:
            # Intentar convertir a float
            return float(value)
        except ValueError:
            # Si hay error, regresa el valor original limpio
            return value
    return value

# Aplicar la función a cada elemento del DataFrame
tabla_df[2] = tabla_df[2].apply(clean_convert)

# Mostrar el DataFrame modificado
print("\nDataFrame Modificado:")
tabla_df


DataFrame Modificado:


,0,2
1,VL 18/04/2024,USD 148.51
2,Cambio del día,-0.72
3,Categoría Morningstar™,RV Global Cap. Flexible
4,ISIN,LU1785301273
5,Patrimonio (Mil) 18/04/2024,USD 573.04
6,Patrimonio Clase (Mil) 18/04/2024,USD 8.69
7,Comisión Máx. Suscripción,-
8,Gastos Corrientes 30/06/2023,1.0


In [15]:
# Encontrar todas las tablas en el documento
tables = soup.find_all('table')

# Lista para almacenar los nombres/identificadores de las tablas
table_names = []

for table in tables:
    # Intentar obtener el atributo 'id'
    if table.has_attr('id'):
        table_names.append(table['id'])
    # Si no hay 'id', intentar obtener el atributo 'class'
    elif table.has_attr('class'):
        # Los atributos de clase se devuelven como una lista, los convertimos a string
        table_names.append(' '.join(table['class']))
    # Si no hay ni 'id' ni 'class', usar un placeholder o revisar otro atributo
    else:
        table_names.append('Unnamed Table')

# Imprimir los nombres de las tablas
print(table_names)

['snapshotTextColor snapshotTextFontStyle snapshotTitleTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewPerformanceTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewCalenderYearReturnsTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewKeyStatsTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewObjectiveTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable TrailingReturnsOverview', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewTrailingReturnsTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewKeyStatsTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable FundManagersOverview', 'snapshotTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewBenchmarkTable2Cols', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewObjectiveTable', 'snapshotTextColor snapshotTextFontStyle snapshotTable overviewPortfolioTable', 'snapshotTextColor s

In [39]:
# Buscar la tabla por clase
table = soup.find('table', class_='overviewKeyStatsTable')

# Extraer las filas de la tabla
rows = table.find_all('tr')

# Lista para almacenar cada fila de datos
data = []

# Iterar sobre las filas para extraer datos de cada celda
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append(cols)  # Añadir los datos de las celdas a la lista `data`

# Crear un DataFrame con los datos recopilados
df = pd.DataFrame(data)

# Opcional: si la primera fila contiene los nombres de las columnas
df.columns = df.iloc[0]  # Establecer la primera fila como nombres de columnas
df = df[1:]  # Quitar la primera fila de los datos

print(df)

0           Estadística Rápida None                 None
1                 VL28/03/2024                USD 197,50
2               Cambio del día                     0,66%
3       Categoría Morningstar™       Alternativo Divisas
4                         ISIN              MT7000022612
5            Patrimonio (Mil)-                         -
6      Patrimonio Clase (Mil)-                         -
7    Comisión Máx. Suscripción                         -
8  Gastos Corrientes10/03/2021                     3,00%


In [41]:
table = soup.find('table', class_='overviewObjectiveTable')

# Extraer las filas de la tabla
rows = table.find_all('tr')

# Lista para almacenar cada fila de datos
data = []

# Iterar sobre las filas para extraer datos de cada celda
for row in rows:
    # Extraer la celda de la fila
    cols = row.find_all('td')
    # Obtener el texto de cada celda y eliminar espacios en blanco extra con strip
    cols_text = [ele.text.strip() for ele in cols]
    data.append(cols_text)

# Crear un DataFrame con los datos recopilados
df = pd.DataFrame(data, columns=["Descripción"])

print(df)

                                         Descripción
0  Objetivo de inversión: 24 Capital Management S...
1  The investment objective of the Fund is to ach...
